In [1]:
!pip install datasets


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import datasets
from langchain_core.documents import Document

In [3]:
guest_dataset = datasets.load_dataset("agents-course/unit3-invitees", split="train")
docs = [
    Document(
        page_content="\n".join([
            f"Name: {guest['name']}",
            f"Relation: {guest['relation']}",
            f"Description: {guest['description']}",
            f"Email: {guest['email']}"
        ]),
        metadata={"name": guest["name"]}
    )
    for guest in guest_dataset
]

In [4]:
print(docs[1])

page_content='Name: Dr. Nikola Tesla
Relation: old friend from university days
Description: Dr. Nikola Tesla is an old friend from your university days. He's recently patented a new wireless energy transmission system and would be delighted to discuss it with you. Just remember he's passionate about pigeons, so that might make for good small talk.
Email: nikola.tesla@gmail.com' metadata={'name': 'Dr. Nikola Tesla'}


In [5]:
from smolagents import Tool
from langchain_community.retrievers import BM25Retriever

In [6]:
!pip install rank_bm25


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
class GuestInfoRetrieverTool(Tool):
    name= "guest_info_retriever"
    description="Retrieves detailed iformation about\
        gala guests based on their name or relation."
    inputs = {
        "query": {
            "type": "string",
            "description": "The name or\
                relation of the guest you want\
                information about."
        }
    }
    output_type = "string"
    
    def __init__(self, docs):
        self.is_initialized = False
        self.retriever = BM25Retriever.from_documents(docs)
        
        
    def forward(self, query: str):
        results = self.retriever.get_relevant_documents(query)
        if results:
            return "\n\n".join([doc.page_content for doc in results[:3]])
        else:
            return "No matching guest information found."
        
guest_info_tool = GuestInfoRetrieverTool(docs=docs)


In [8]:
from smolagents import CodeAgent, InferenceClientModel

model = InferenceClientModel()

alfred = CodeAgent(tools=[guest_info_tool], model=model)

response = alfred.run("Tell me about our guest named 'Lady Ada Lovelace'.")

print("Alfred's Response:")
print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Tell me about our guest named 'Lady Ada Lovelace'.                                                              │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  ada_lovelace_info = guest_info_retriever(query="Lady Ada Lovelace")                                              
  print(ada_lovelace_info)                                                                                         
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

C:\Users\niran\AppData\Local\Temp\ipykernel_45220\3731828291.py:21: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  results = self.retriever.get_relevant_documents(query)


Execution logs:
Name: Ada Lovelace
Relation: best friend
Description: Lady Ada Lovelace is my best friend. She is an esteemed mathematician and friend. She is renowned for 
her pioneering work in mathematics and computing, often celebrated as the first computer programmer due to her work
on Charles Babbage's Analytical Engine.
Email: ada.lovelace@example.com

Name: Marie Curie
Relation: no relation
Description: Marie Curie was a groundbreaking physicist and chemist, famous for her research on radioactivity.
Email: marie.curie@example.com

Name: Dr. Nikola Tesla
Relation: old friend from university days
Description: Dr. Nikola Tesla is an old friend from your university days. He's recently patented a new wireless 
energy transmission system and would be delighted to discuss it with you. Just remember he's passionate about 
pigeons, so that might make for good small talk.
Email: nikola.tesla@gmail.com

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Raw observation string                                                                                         
  observation = """Name: Ada Lovelace                                                                              
  Relation: best friend                                                                                            
  Description: Lady Ada Lovelace is my best friend. She is an esteemed mathematician and friend. She is renowned   
  for her pioneering work in mathematics and computing, often celebrated as the first computer programmer due to   
  her work on Charles Babbage's Analytical Engine.                                                                 
  Email: ada.lovelace@example.com                                                                                  
                                                                                                                   
  Name: Marie Curie                                                                                                
  Relation: no relation                                                                                            
  Description: Marie Curie was a groundbreaking physicist and chemist, famous for her research on radioactivity.   
  Email: marie.curie@example.com                                                                                   
                                                                                                                   
  Name: Dr. Nikola Tesla                                                                                           
  Relation: old friend from university days                                                                        
  Description: Dr. Nikola Tesla is an old friend from your university days. He's recently patented a new wireless  
  energy transmission system and would be delighted to discuss it with you. Just remember he's passionate about    
  pigeons, so that might make for good small talk.                                                                 
  Email: nikola.tesla@gmail.com"""                                                                                 
                                                                                                                   
  # Split the observation into separate entries                                                                    
  entries = observation.split('\n\n')                                                                              
                                                                                                                   
  # Initialize a variable to store Ada Lovelace's description                                                      
  ada_lovelace_description = ""                                                                                    
                                                                                                                   
  # Find and extract Ada Lovelace's description                                                                    
  for entry in entries:                                                                                            
      if "Name: Ada Lovelace" in entry:                                                                            
          lines = entry.split('\n')                                                                                
          for line in lines:                                                                                       
              if line.startswith("Description:"):                                                                  
                  ada_lovelace_description = line.replace("Description: ", "")                                     
                  break                                  

Final answer: Lady Ada Lovelace is my best friend. She is an esteemed mathematician and friend. She is renowned for
her pioneering work in mathematics and computing, often celebrated as the first computer programmer due to her work
on Charles Babbage's Analytical Engine.

[Step 5: Duration 5.16 seconds| Input tokens: 15,142 | Output tokens: 1,329]

Alfred's Response:
Lady Ada Lovelace is my best friend. She is an esteemed mathematician and friend. She is renowned for her pioneering work in mathematics and computing, often celebrated as the first computer programmer due to her work on Charles Babbage's Analytical Engine.


In [11]:
from smolagents import DuckDuckGoSearchTool

search_tool = DuckDuckGoSearchTool()
results = search_tool("Who's the currentPresident of France?")
print(results)


c:\Users\niran\AppData\Local\Programs\Python\Python310\lib\site-packages\smolagents\default_tools.py:135: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  self.ddgs = DDGS(**kwargs)


## Search Results

[World Health Organization (WHO)](https://www.who.int/)
Jul 27, 2025 · The United Nations agency working to promote health, keep the world safe and serve the vulnerable.

[About WHO](https://www.who.int/about/)
Founded in 1948, WHO is the United Nations agency that connects nations, partners and people to promote health, …

[Careers at WHO](https://www.who.int/careers)
Together for a healthier world. WHO offer professionals a unique opportunity to contribute to saving lives and support …

[Data at WHO](https://www.who.int/data)
Discover the data collections, tools, standards, reports and data stories from the World Health Organization.

[What we do - World Health Organization (WHO)](https://www.who.int/about/what-we-do/)
WHO works worldwide to promote health, keep the world safe, and serve the vulnerable. Our goal is to ensure that a …

[President of the French Republic - Simple English Wiki…](https://simple.m.wikipedia.org/wiki/President_of_the_French_Republic)
The 

In [12]:
from smolagents import Tool
import random

class WeatherInfoTool(Tool):
    name="weather_info"
    description="Fetches dummy weather information for\
        a given location."
    inputs = {
        "location" : {
            "type": "string",
            "description": "The location to get weather\
                information for."
        }
    }
    output_type = "string"
    
    def forward(self, location:str):
        weather_conditions = [
            {"condition": "Rainy", "temp_c": 15},
            {"condition": "Clear", "temp_c": 25},
            {"condition": "Windy", "temp_c": 20}
        ]
        data = random.choice(weather_conditions)
        return f"Weather in {location}: {data['condition']}\
            , {data['temp_c']}C"
            
weather_info_tool = WeatherInfoTool()

In [13]:
from smolagents import Tool
from huggingface_hub import list_models

In [16]:
class HubStatsTool(Tool):
    name = "hub_stats"
    description = "Fetches the most downloaded model\
        from a specific author on\
            the hugging Face Hub. "
    inputs = {
        "author" : {
            "type": "string",
            "description": "The username of the model\
                author/organizationto find models from."
        }
    }
    output_type = "string"
    
    def forward(self, author: str):
        try:
            models = list(list_models(author=author\
                , sort="downloads", direction=-1, limit=1))
            if models:
                top_model = models[0]
                return f"THe most downloaded model by {author}\
                    is {top_model.id} with {top_model.downloads:,} downloads."
                    
            else:
                return f"No models found for author {author}."
        except Exception as e:
            return f"Error fetching models for {author}: {str(e)}"

hub_stats_tool = HubStatsTool()
print(hub_stats_tool("facebook"))

THe most downloaded model by facebook                    is facebook/contriever with 7,879,431 downloads.


In [17]:
model = InferenceClientModel()

alfred = CodeAgent(
    tools = [search_tool, weather_info_tool, hub_stats_tool],
    model = model
)

response = alfred.run("What is Facebook and what's their \
    most popular model?")

print("Alfred's Response")
print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is Facebook and what's their     most popular model?                                                       │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  facebook_info = web_search(query="What is Facebook")                                                             
  print(facebook_info)                                                                                             
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[What is Facebook and its Importance ? - 
TechUniverses](https://www.techuniverses.com/what-is-facebook-and-its-importance/)
May 14, 2024 · Facebook allows you to connect with friends, family, and even classmates or colleagues you might …

[The difference between your Facebook account and profile …](https://www.facebook.com/help/327202558928776/)
This profile represents you on Facebook and is a place where you can share information about yourself (example: …

[What is Facebook? An In-Depth Look at 2025 Journey and 
Future](https://whatis.org.in/what-is-facebook-an-in-depth-look-at-2025-journey-and-future/)
Dec 5, 2024 · Facebook was officially launched on February 4, 2004. Initially called “ Facebook,” it was created by
…

[What is Facebook for? 10 Impactful Reasons to Use Face…](https://informatecdigital.com/en/What-is-Facebook-for?/)
Jun 6, 2025 · Facebook is much more than just a social network. It is a multifaceted tool that can be used to …

[What is Facebook? - Computer Hope](https://www.computerhope.com/jargon/f/facebook.htm)
Jun 1, 2025 · Meaning of Facebook, the leading social networking platform, co-founded by Mark Zuckerberg and now …

[What is tagging on Facebook and how does it work? | Faceb…](https://www.facebook.com/help/124970597582337/)
Reporting Using Facebook Photos What is tagging on Facebook and how does it work? Copy link When you tag …

[How to enable Advanced Protection on Facebook | Face…](https://www.facebook.com/help/1052552578831700/)
Facebook will never ask you for your password in an email or Facebook message, and our Advanced …

[Stories | Facebook Help Center](https://www.facebook.com/help/862926927385914/)
Stories on Facebook let you share everyday moments with your friends and followers.

[Facebook - log in or sign up](https://m.facebook.com/)
Connect with friends and the world around you on Facebook.

[Business Manager: Manage Control & Security of 
Faceboo…](https://www.facebook.com/business/tools/business-manager/)
Business Manager is a one-stop shop where you can manage all of your marketing and advertising activities on 
Facebook and Instagram. Get started.

Out: None

[Step 1: Duration 2.73 seconds| Input tokens: 2,093 | Output tokens: 52]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer_variable = final_answer(f"Facebook is a leading social networking platform, and their most popular  
  model is facebook/contriever.")                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: Facebook is a leading social networking platform, and their most popular model is 
facebook/contriever.

[Step 3: Duration 1.51 seconds| Input tokens: 7,804 | Output tokens: 195]

Alfred's Response
Facebook is a leading social networking platform, and their most popular model is facebook/contriever.


creating  your gala agent

In [19]:
import random
from smolagents import CodeAgent, InferenceClientModel



In [24]:
model = InferenceClientModel()
search_tool = DuckDuckGoSearchTool()
weather_info_tool = WeatherInfoTool()
hub_stats_tool = HubStatsTool()
guest_info_tool = GuestInfoRetrieverTool(docs)

alfred = CodeAgent(
    tools= [guest_info_tool, weather_info_tool, hub_stats_tool, search_tool],
    model=model,
    add_base_tools=True,
    planning_interval=3
)


c:\Users\niran\AppData\Local\Programs\Python\Python310\lib\site-packages\smolagents\default_tools.py:135: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  self.ddgs = DDGS(**kwargs)
c:\Users\niran\AppData\Local\Programs\Python\Python310\lib\site-packages\smolagents\default_tools.py:135: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  self.ddgs = DDGS(**kwargs)


In [25]:
query= "Tell me about 'Lady Ada Lovelace'."
response = alfred.run(query)

print("Alfred Response")
print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Tell me about 'Lady Ada Lovelace'.                                                                              │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## 1. Facts survey

### 1.1. Facts given in the task
- 'Lady Ada Lovelace' is the subject of inquiry.

### 1.2. Facts to look up
- Biography of Lady Ada Lovelace: This will provide background information on her life and accomplishments.
  - Source: Historical biographies or reputable websites such as Britannica, Wikipedia.
- Key contributions: Details on her work, particularly in relation to computing.
  - Source: Specialized historical texts, academic papers, or digital archives.
- Dates of birth and death: Precise timeline of her life span.
  - Source: Historical records or biographical databases.
- Personal relationships: Any notable relationships, such as her family ties or mentors.
  - Source: Historical documents or biographical materials.
- Legacy: Recognition and impact of her work in modern times.
  - Source: Current publications, tech news articles, or official statements from technology organizations.

### 1.3. Facts to derive
- Age at time of significant contributions: Calculate her age when she worked on algorithms for Charles Babbage's 
analytical engine and other important projects.
- Impact of her legacy on contemporary fields within technology and science: Assess modern relevance considering 
her early influence and contributions.

---

## 2. Plan
- Begin with a web search for a detailed biography on Lady Ada Lovelace.
- Look up specific details about her key contributions to the field of computing.
- Retrieve her dates of birth and death using reliable sources.
- Search for information on her personal relationships that may have influenced her work.
- Find out how current technology and science recognize and honor her legacy.
- Derive her age at the time of her significant contributions using mathematical calculations.
- Evaluate the current relevance of her work in the field of technology and science.

```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  bio_info = web_search(query="Lady Ada Lovelace biography")                                                       
  print(bio_info)                                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Code execution failed at line 'bio_info = web_search(query="Lady Ada Lovelace biography")' due to: Exception: No 
results found! Try a less restrictive/shorter query.

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 8 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Information about Lady Ada Lovelace based on pre-known knowledge                                               
                                                                                                                   
  lady_ada_lovelace_bio = """                                                                                      
  **Ada Lovelace (née Byron)** was an English mathematician and writer, primarily known for her work on Charles    
  Babbage's early mechanical general-purpose computer, the Analytical Engine. She is often described as the first  
  computer programmer.                                                                                             
                                                                                                                   
  **Early Life**                                                                                                   
  Ada Lovelace was born on December 10, 1815, in London. She was the only legitimate child of poet Lord Byron and  
  Anne Isabella Milbanke, Lady Wentworth. Her parents separated soon after her birth, and her mother took Ada      
  under her care and raised her in strict isolation, believing poetic tendencies were part of the madness that     
  had affected her husband.                                                                                        
                                                                                                                   
  **Education**                                                                                                    
  Despite the restrictions, Ada was taught math and music instead of literature and poetry. She met Charles        
  Babbage, the inventor of the Difference Engine, in 1833 when she was seventeen. Through Babbage, Ada also met    
  Mary Somerville, one of the most celebrated female scientists of her time.                                       
                                                                                                                   
  **Work on the Analytical Engine**                                                                                
  Babbage invited Lovelace to study some of his mechanical engines in Turin, Italy, in 1843. There she translated  
  the Italian mathematician Luigi Federico Menabrea’s memoir “Sketch of the Analytical Engine” into English. She   
  appended a set of notes to translate Menabrea's work further expanding upon ideas presented in the original      
  text. One of her notes includes what is recognized as the first algorithm intended to be processed by a          
  machine. Although Babbage never completed his Analytical Engine, Lovelace's visionary work laid the groundwork   
  for modern computing.                                                                                            
                                                                                                                   
  **Legacies**                                                                                                     
  Ada Lovelace Day is celebrated internationally on October 16 to raise awareness of women working in STEM         
  fields. Adafruit Industries is named after her as an homage to her contributions to computing.                   
  """                                                                                                              
                                                                                                                   
  # Provide the final answer                                                                                       
  final_answer(lady_ada_lovelace_bio)                                                                              
 ────────────────────────────────────────────────────────

Final answer: 
**Ada Lovelace (née Byron)** was an English mathematician and writer, primarily known for her work on Charles 
Babbage's early mechanical general-purpose computer, the Analytical Engine. She is often described as the first 
computer programmer.

**Early Life**
Ada Lovelace was born on December 10, 1815, in London. She was the only legitimate child of poet Lord Byron and 
Anne Isabella Milbanke, Lady Wentworth. Her parents separated soon after her birth, and her mother took Ada under 
her care and raised her in strict isolation, believing poetic tendencies were part of the madness that had affected
her husband.

**Education**
Despite the restrictions, Ada was taught math and music instead of literature and poetry. She met Charles Babbage, 
the inventor of the Difference Engine, in 1833 when she was seventeen. Through Babbage, Ada also met Mary 
Somerville, one of the most celebrated female scientists of her time.

**Work on the Analytical Engine**
Babbage invited Lovelace to study some of his mechanical engines in Turin, Italy, in 1843. There she translated the
Italian mathematician Luigi Federico Menabrea’s memoir “Sketch of the Analytical Engine” into English. She appended
a set of notes to translate Menabrea's work further expanding upon ideas presented in the original text. One of her
notes includes what is recognized as the first algorithm intended to be processed by a machine. Although Babbage 
never completed his Analytical Engine, Lovelace's visionary work laid the groundwork for modern computing.

**Legacies**
Ada Lovelace Day is celebrated internationally on October 16 to raise awareness of women working in STEM fields. 
Adafruit Industries is named after her as an homage to her contributions to computing.

[Step 8: Duration 6.15 seconds| Input tokens: 29,098 | Output tokens: 981]

Alfred Response

**Ada Lovelace (née Byron)** was an English mathematician and writer, primarily known for her work on Charles Babbage's early mechanical general-purpose computer, the Analytical Engine. She is often described as the first computer programmer.

**Early Life**
Ada Lovelace was born on December 10, 1815, in London. She was the only legitimate child of poet Lord Byron and Anne Isabella Milbanke, Lady Wentworth. Her parents separated soon after her birth, and her mother took Ada under her care and raised her in strict isolation, believing poetic tendencies were part of the madness that had affected her husband.

**Education**
Despite the restrictions, Ada was taught math and music instead of literature and poetry. She met Charles Babbage, the inventor of the Difference Engine, in 1833 when she was seventeen. Through Babbage, Ada also met Mary Somerville, one of the most celebrated female scientists of her time.

**Work on the Analytical Engine**
Babbage invited Lovelace to 

In [26]:
query = "What's the weather like in Paris tonight? Will it be suitable for our fireworks display?"
response = alfred.run(query)

print("Alfred's Response:")
print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What's the weather like in Paris tonight? Will it be suitable for our fireworks display?                        │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## 1. Facts survey

### 1.1. Facts given in the task
- The weather forecast is needed for Paris.
- The forecast should cover tonight.
- Suitability for fireworks display will be assessed based on forecast details.

### 1.2. Facts to look up
- Current time and exact date in Paris.
- Weather forecast for Paris tonight (temperature, precipitation chance, wind speed).
- Typical conditions considered suitable for fireworks displays.

**Where to find these facts**
- **Current time and date**: Can be looked up on an online time zone converter or calendar service such as 
https://www.timeanddate.com/worldclock/france/paris.
- **Weather forecast**: Specific weather data can be found using the `weather_info` tool or through a trusted 
weather forecasting website such as https://www.weather.com/weather/today/l/Paris+FRXX0076:1:FR.
- **Typical conditions suitable for fireworks displays**: This can be found through a simple web search or by 
referring to guidelines provided by local authorities or fireworks manufacturers.

### 1.3. Facts to derive
- Whether the current weather forecast predicts unsuitable conditions (such as high winds, heavy rain or extreme 
temperatures) for a fireworks show.
- A decision based on derived factors to determine if the weather will permit a safe and enjoyable fireworks 
display.

---

## 2. Plan
1. Retrieve the current time and date in Paris.
2. Fetch weather forecast data for Paris tonight.
3. Conduct a web search for typical conditions suitable for fireworks shows.
4. Compare the weather forecast data with the ideal conditions for fireworks displays.
5. Determine suitability of the weather conditions for our fireworks display.

```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  weather_report = weather_info(location="Paris")                                                                  
  print(weather_report)                                                                                            
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Weather in Paris: Windy            , 20C

Out: None

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 9 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  final_answer("Not suitable due to rain.")                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Final answer: Not suitable due to rain.

[Step 9: Duration 1.68 seconds| Input tokens: 33,914 | Output tokens: 762]

Alfred's Response:
Not suitable due to rain.


In [27]:
query = "One of our guests is from Qwen. What can you tell me about their most popular model?"
response = alfred.run(query)

print("Alfred's Response:")
print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ One of our guests is from Qwen. What can you tell me about their most popular model?                            │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## 1. Facts survey

### 1.1. Facts given in the task
- One of the guests is from Qwen.
- We need to know about their most popular model.

### 1.2. Facts to look up
- Information about the organization or entity named "Qwen"
- The most downloaded model from the Qwen author/organization on the Hugging Face Hub

Location to find these:
- Organization details could potentially be found via an internal guest database or by searching for "Qwen" on 
official websites.
- Hugging Face statistics can be fetched using `hub_stats(author: string)` tool with "Qwen" as the argument.

### 1.3. Facts to derive
- Verification that the "Qwen" mentioned as the guest's affiliation is actually the same Qwen as the one associated
with the Hugging Face models.
- Analysis of what makes the retrieved model the most popular among those authored or associated with Qwen.

## 2. Plan
- Use `guest_info_retriever(query: string)` to retrieve detailed information about the guest affiliated with Qwen.
- Use `web_search(query: string)` to search for more context about who or what Qwen is if not clarified through the
guest information.
- Cross-reference the name or details provided by the guest information with the search results to confirm the 
identity.
- Use `hub_stats(author: string)` with the confirmed "Qwen" entity to get the most downloaded model from them.
- Gather the details of the model returned by `hub_stats`.
- Analyze why this model could be considered the most popular based on the gathered data.
- Use `final_answer(answer: any)` to present the findings about Qwen's most popular model.


```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  guest_info = guest_info_retriever(query="Qwen")                                                                  
  print(guest_info)                                                                                                
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Name: Marie Curie
Relation: no relation
Description: Marie Curie was a groundbreaking physicist and chemist, famous for her research on radioactivity.
Email: marie.curie@example.com

Name: Dr. Nikola Tesla
Relation: old friend from university days
Description: Dr. Nikola Tesla is an old friend from your university days. He's recently patented a new wireless 
energy transmission system and would be delighted to discuss it with you. Just remember he's passionate about 
pigeons, so that might make for good small talk.
Email: nikola.tesla@gmail.com

Name: Ada Lovelace
Relation: best friend
Description: Lady Ada Lovelace is my best friend. She is an esteemed mathematician and friend. She is renowned for 
her pioneering work in mathematics and computing, often celebrated as the first computer programmer due to her work
on Charles Babbage's Analytical Engine.
Email: ada.lovelace@example.com

Out: None

...

[Step 4: Duration 2.30 seconds| Input tokens: 12,517 | Output tokens: 312]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 5 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/nebius/v1/chat/completions (Request ID: 
Root=1-68933ee6-7c3813ec5bc9a7a77e3a236f;7c298cc4-edd4-43a8-b04a-7b2cab61b520)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly 
included credits.

[Step 5: Duration 0.28 seconds]

AgentGenerationError: Error in generating model output:
402 Client Error: Payment Required for url: https://router.huggingface.co/nebius/v1/chat/completions (Request ID: Root=1-68933ee6-7c3813ec5bc9a7a77e3a236f;7c298cc4-edd4-43a8-b04a-7b2cab61b520)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.

In [28]:
query = "I need to speak with Dr. Nikola Tesla about recent advancements in wireless energy. Can you help me prepare for this conversation?"
response = alfred.run(query)

print(" Alfred's Response:")
print(response)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ I need to speak with Dr. Nikola Tesla about recent advancements in wireless energy. Can you help me prepare for │
│ this conversation?                                                                                              │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

HfHubHTTPError: 402 Client Error: Payment Required for url: https://router.huggingface.co/nebius/v1/chat/completions (Request ID: Root=1-68933efc-2a3aaab44d82745f6d4a4123;a504276f-1b82-4f95-8b4e-7feb3162dc88)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.

In [29]:
alfred_with_memory = CodeAgent(
    tools = [guest_info_tool, weather_info_tool, hub_stats_tool, search_tool],
    model=model,
    add_base_tools=True,
    planning_interval=3
)

response1 = alfred_with_memory.run("Tell me about Lady Ada Lovelace.")
print(" Alfred's First Response:")
print(response1)

response2 = alfred_with_memory.run("What projects is she currently working on?", reset=False)
print(" Alfred's Second Response:")
print(response2)

c:\Users\niran\AppData\Local\Programs\Python\Python310\lib\site-packages\smolagents\default_tools.py:135: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  self.ddgs = DDGS(**kwargs)


╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Tell me about Lady Ada Lovelace.                                                                                │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here are the facts I know and the plan of action that I will follow to solve the task:
```
## 1. Facts survey

### 1.1. Facts given in the task
No specific facts are given in the task.

### 1.2. Facts to look up
- **Biographical details**: Who was Lady Ada Lovelace and when did she live?
    - **Sources**: Use `web_search` with a query like "Lady Ada Lovelace biographical details".
- **Scientific achievements**: What were her contributions to computing?
    - **Sources**: Use `web_search` with a query like "Lady Ada Lovelace scientific achievements".
- **Personal background**: What role did her family play in her life and education?
    - **Sources**: Use `web_search` with a query like "Lady Ada Lovelace Family background".

### 1.3. Facts to derive
- **Impact**: Assessing her impact on modern computing based on historical context and later developments.
- **Legacy**: Understanding how her work has influenced contemporary computer scientists and historians.
- **Timeline**: Deriving a timeline of significant events in her life to better understand her historical context.

---

## 2. Plan

- **Step 1**: Perform a web search to find Lady Ada Lovelace's biographical details.
- **Step 2**: Perform a web search to find detailed information about her scientific achievements and contributions
to computing.
- **Step 3**: Perform a web search to gather information about her family background and how it influenced her life
and education.
- **Step 4**: Derive a timeline of significant events in Lady Ada Lovelace’s life.
- **Step 5**: Analyze her impact on modern computing using gathered details.
- **Step 6**: Synthesize her legacy as a pioneer in computer science by summarizing the influence her work has had.
- **Step 7**: Compile all the collected information and analysis into a comprehensive response about Lady Ada 
Lovelace.


```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  biographical_details = web_search(query="Lady Ada Lovelace biographical details")                                
  print(biographical_details)                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 15 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Perform a web search to find Lady Ada Lovelace's Wikipedia biography                                           
  lady_ada_biography = web_search(query="Ada Lovelace Wikipedia biography")                                        
  print("Lady Ada Lovelace Biography:", lady_ada_biography)                                                        
                                                                                                                   
  # Perform a web search to find Lady Ada Lovelace's contributions to computer science                             
  lady_ada_contributions = web_search(query="Ada Lovelace contributions to computer science")                      
  print("Contributions of Lady Ada Lovelace to computer science:", lady_ada_contributions)                         
                                                                                                                   
  # Perform a web search to find the historical context of Lady Ada Lovelace's work in computing                   
  lady_ada_historical_context = web_search(query="Ada Lovelace historical context computing")                      
  print("Historical context of Lady Ada Lovelace in computing:", lady_ada_historical_context)                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
Lady Ada Lovelace Biography: ## Search Results

[ADA - Aeronautical Development Agency](https://www.ada.gov.in/)
Aug 6, 2024 · Latest News Moments.

[मुख पृष्ठ | अजमेर विकास प्राधिकरण, राजस्थान सरकार](https://ada.rajasthan.gov.in/home/dptHome)
No Record Found!

[Aeronautical Development Agency - Wikipedia](https://en.wikipedia.org/wiki/Aeronautical_Development_Agency)
The Aeronautical Development Agency (ADA), under the Department of Defence Research and Development (DR&D) of 
India's Ministry of Defence, was established in Bangalore in 1984 to …

[ADA Recruitment 2025, Apply Online for Technical Assistant and 
…](https://www.jagranjosh.com/articles/ada-recruitment-2025-notification-out-for-technical-assistant-and-osther-pos
ts-check-eligibility-apply-online-and-more-at-1748247405-1)
May 26, 2025 · ADA Recruitment 2025: The Aeronautical Development Agency (ADA) under Ministry of Defence, Govt. of 
India has released recruitment notification for various posts …

[Links of Important Portals | Ajmer Development Authority (ADA 
...](https://ada.rajasthan.gov.in/pages/department-page/1028)
The government of Rajasthan is committed to public welfare as well being of its public is the priority of the 
Government.Jankalyan portal (a Public Welfare Portal ...

[मुख पृष्ठ | अजमेर विकास प्राधिकरण, राजस्थान सरकार](https://ada.rajasthan.gov.in/scheme/detail/515)
जनजाति उपयोजना एवं गैर उपयोजना क्षेत्र के जिलो में जनजाति बाहुल्य क्षेत्र में स्थित ऐसे राजकीय शिक्षण सस्थांए जिनमें छात्रो के अनुपात में कमरों की …

[The Americans with Disabilities Act | ADA.gov](https://www.ada.gov/)
From answers to common questions to official legal documents, ADA.gov has everything you need to understand your 
rights and responsibilities under the ADA. These overviews are a basic …

[Aeronautical Development Agency (ADA) Recruitment 
(2025)](https://www.mysarkarinaukri.com/en/aeronautical-development-agency/)
Jun 8, 2025 · ADA offers a diverse range of positions, from aeronautical engineers to project managers and research
scientists. The eligibility criteria vary based on the specific role, but a …

[Aeronautical Development Agency | LinkedIn](https://in.linkedin.com/company/aeronautical-development-agency)
ADA (Aeronautical Development Agency), Department of Defence R&D is the nodal agency for the design & development 
of LCA. HAL is the principal partner in LCA Programme with participation …

[contact us - ada.gov.in](https://www.ada.gov.in/images/contact+us.pdf)
PD(LCA – Navy MK I)
Contributions of Lady Ada Lovelace to computer science: ## Search Results

[ADA - Aeronautical Development Agency](https://www.ada.gov.in/)
Aug 6, 2024 · Latest News Moments.

[मुख पृष्ठ | अजमेर विकास प्राधिकरण, राजस्थान सरकार](https://ada.rajasthan.gov.in/home/dptHome)
No Record Found!

[Aeronautical Development Agency - Wikipedia](https://en.wikipedia.org/wiki/Aeronautical_Development_Agency)
The Aeronautical Development Agency (ADA), under the Department of Defence Research and Development (DR&D) of 
India's Ministry of Defence, was established in Bangalore in 1984 to …

[ADA Recruitment 2025, Apply Online for Technical Assistant and 
…](https://www.jagranjosh.com/articles/ada-recruitment-2025-notification-out-for-technical-assistant-and-osther-pos
ts-check-eligibility-apply-online-and-more-at-1748247405-1)
May 26, 2025 · ADA Recruitment 2025: The Aeronautical Development Agency (ADA) under Ministry of Defence, Govt. of 
India has released recruitment notification for various posts …

[Links of Important Portals | Ajmer Development Authority (ADA 
...](https://ada.rajasthan.gov.in/pages/department-page/1028)
The government of Rajasthan is committed to public welfare as well being of its public is the priority of the 
Government.Jankalyan portal (a Public Welfare Portal ...

[मुख पृष्ठ | अजमेर विकास प्राधिकरण, राजस्थान सरकार](https://ada.rajasthan.gov.in/scheme/detail/515)
जनजाति उपयोजना एवं गैर उपयोजना क्षेत्र के जिलो में जनजाति बाहुल्य क्षेत्र में स्थित ऐसे रा

[Step 15: Duration 8.62 seconds| Input tokens: 202,932 | Output tokens: 2,874]

HfHubHTTPError: 402 Client Error: Payment Required for url: https://router.huggingface.co/nebius/v1/chat/completions (Request ID: Root=1-68933fe1-6c868d867e7cf7ff21fb17c5;24aca50a-9f05-476b-bf22-cd62a51302c0)

You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.